# Detecting wild reindeers in CWD contaminated area

In [1]:
from ipyleaflet import Marker, Icon, Map, Polygon, GeoJSON, LayerGroup, DrawControl, Popup
import json
import os
import requests
import numpy as np
from random import randrange, uniform
from ipywidgets import interact, HTML
from shapely.geometry import shape, Point

n=15
center = (60.88, 7.58)
lat_start = 60.68
lon_start = 7.58
scale = .05


m = Map(center=center, zoom=8)
mark = Marker(location=[lat_start, lon_start], rotation_angle=0, rotation_origin='22px 94px') 
group = LayerGroup(layers=(mark,))
m.add_layer(group)

In [2]:
with open('zone_1.json') as f:
    zone1_data = json.load(f)
with open('zone_2.json') as f:
    zone2_data = json.load(f)
layer_zone1 = GeoJSON(data=zone1_data)
m.add_layer(layer_zone1)
layer_zone2 = GeoJSON(data=zone2_data)
m.add_layer(layer_zone2)    

In [3]:
ninside=0
def coords(time):
#    try:
    try:
        group.clear_layers()
    except:
        pass
        

    i=n
    ninside=0
    while i > 0:


        la = lat_start + uniform(-scale, scale) + uniform(-0.06,0.06)
        lo = lon_start + uniform(-scale, scale)+ uniform(-0.06,0.06)
        point = Point(lo,la)
        mark = Marker(location=[la, lo], rotation_angle=0, rotation_origin='22px 94px') 
        group.add_layer(mark)     
        
        outside=True
        for s in zone1_data['features']:
            polygon = shape(s['geometry'])
            
            if point.within(polygon):
                print("INSIDE: Raise alarm")
                outside=False
                ninside = ninside+1
        
        i -=1

    if ninside>0:
        alarm = HTML()
        alarm.value = "ALARM: " + str(ninside) + " reindeers in the CONTAMINATED ZONE - SHOOT IT!!!"
        print("ALARM")

    # Popup with a given location on the map:
        popup = Popup(
            location=(60.77, 7.65),
            child=alarm,
            close_button=True,
            auto_close=False,
            close_on_escape_key=False
        )
        m.add_layer(popup)
    

#coords(0.0)


with open('nordfjella.json', 'r') as f:
  data = json.load(f)
geo_json = GeoJSON(data=data, style = {'color': 'green', 'opacity':1, 'weight':1.9, 'dashArray':'9', 'fillOpacity':0.1})
m.add_layer(geo_json)

polygon = Polygon(
    locations=[(60.68753821944088,7.752502404083099), (60.75302523120072,7.809503970433292), 
               (60.80428536387097,7.559768404156764), (60.73925825494969,7.509379087922763)],
    color="red",
    fill_color="red"
)

m.add_layer(polygon);


interact(coords, time=0.0);
m

interactive(children=(FloatSlider(value=0.0, description='time', max=1.0), Output()), _dom_classes=('widget-in…

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [4]:
#Add a marker when you see a reindeer 
dc = DrawControl(circle={'shapeOptions': {'color': 'magenta'}}, circleMarker={}, polyline={}, polygon={})

def handle_draw(self, action, geo_json):
    print(action)
    print(geo_json)

dc.on_draw(handle_draw)
m.add_control(dc)

In [5]:
legend = HTML()
legend.value = "Contaminated zone"

    # Popup with a given location on the map: 
popup = Popup(
        location=(60.8192, 8.2856),
        child=legend,
        close_button=False,
        auto_close=False,
        close_on_escape_key=False
    )
m.add_layer(popup)